Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en tres partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

- [Actividad 1: Redes Densas](#actividad_1): 4 pts
    - Correcta normalización: máximo de 0.25 pts
    - [Cuestión 1](#1.1): 1 pt
    - [Cuestión 2](#1.2): 1 pt
    - [Cuestión 3](#1.3): 0.5 pts
    - [Cuestión 4](#1.4): 0.25 pts
    - [Cuestión 5](#1.5): 0.25 pts
    - [Cuestión 6](#1.6): 0.25 pts
    - [Cuestión 7](#1.7): 0.25 pts
    - [Cuestión 8](#1.8): 0.25 pts


- [Actividad 2: Redes Convolucionales](#actividad_2): 4 pts
    - [Cuestión 1](#2.1): 1 pt
    - [Cuestión 2](#2.2): 1.5 pt
    - [Cuestión 3](#2.3): 0.5 pts
    - [Cuestión 4](#2.4): 0.25 pts
    - [Cuestión 5](#2.5): 0.25 pts
    - [Cuestión 6](#2.6): 0.25 pts
    - [Cuestión 7](#2.7): 0.25 pts
    
    
- [Actividad 3: Redes Recurrentes](#actividad_3): 2 pts
    - [Cuestión 1](#3.1): 0.5 pt
    - [Cuestión 2](#3.2): 0.5 pt
    - [Cuestión 3](#3.3): 0.5 pts
    - [Cuestión 4](#3.4): 0.25 pts
    - [Cuestión 5](#3.5): 0.25 pts



In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
tf.random.set_seed(0)

<a name='actividad_1'></a>
# Actividad 1: Redes Densas

Para esta primera actividad vamos a utilizar el [wine quality dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality). Con el que trataremos de predecir la calidad del vino.

La calidad del vino puede tomar valores decimales (por ejemplo 7.25), independientemente de que en el dataset de entrenamiento sean números enteros. Por lo tanto, el problema es una `regresión`.

**Puntuación**:

Normalizar las features correctamente (x_train, x_test): 0.25 pts , se pueden normalizar con el [Normalization layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Normalization) de Keras. Ejemplo de uso: [Introduction_to_RNN_Time_Series](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)

```python
tf.keras.layers.experimental.preprocessing.Normalization(
    axis=-1, dtype=None, mean=None, variance=None, **kwargs
)
```

- Correcta normalización: máximo de 0.25 pts
- [Cuestión 1](#1.1): 1 pt
- [Cuestión 2](#1.2): 1 pt
- [Cuestión 3](#1.3): 0.5 pts
- [Cuestión 4](#1.4): 0.25 pts
- [Cuestión 5](#1.5): 0.25 pts
- [Cuestión 6](#1.6): 0.25 pts
- [Cuestión 7](#1.7): 0.25 pts
- [Cuestión 8](#1.8): 0.25 pts



In [2]:
# Descargar los datos con pandas
df_red = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv',
    sep=';'
)
df_white = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv',
    sep=';'
)
df = pd.concat([df_red, df_white])

df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
feature_names = [
    'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
    'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol'
]


# separar features y target
y = df.pop('quality').values
X = df.copy().values

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

print('x_train, y_train shapes:', x_train.shape, y_train.shape)
print('x_test, y_test shapes:', x_test.shape, y_test.shape)
print('Some qualities: ', y_train[:5])

x_train, y_train shapes: (4872, 11) (4872,)
x_test, y_test shapes: (1625, 11) (1625,)
Some qualities:  [6 7 8 5 6]


In [5]:
## Si quiere, puede normalizar las features
norm = tf.keras.layers.experimental.preprocessing.Normalization(
    axis=-1, mean=None, variance=None,
)
norm.adapt(x_train)
x_train = norm(x_train)

print(x_train)

tf.Tensor(
[[-0.08802134 -0.72820073  0.0051775  ...  0.9511134  -1.0093288
  -0.7528128 ]
 [-0.7853837  -1.090723    0.07390193 ...  0.6399133  -0.47885838
   0.34073868]
 [-0.3979602  -0.60736    -0.13227095 ... -1.1028085   0.38315573
   0.6772167 ]
 ...
 [-0.3204752  -0.84904146  1.8607332  ... -1.0405685  -1.0093288
  -1.1734098 ]
 [-0.47544482 -0.72820073  0.34879905 ...  0.9511134   0.11792079
  -0.1639772 ]
 [ 1.1517338  -0.72820073  0.69242036 ... -1.4140086  -0.6777848
  -0.41633555]], shape=(4872, 11), dtype=float32)


<a name='1.1'></a>
## Cuestión 1: Cree un modelo secuencial que contenga 4 capas ocultas(hidden layers), con más de 60 neuronas  por capa, sin regularización y obtenga los resultados.

Puntuación:
- Obtener el modelo correcto: 0.8 pts
- Compilar el modelo: 0.1pts
- Acertar con la función de pérdida: 0.1 pts

In [18]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(11,),  name='hidden_layer_1'))  # Capa oculta 1
model.add(tf.keras.layers.Dense(64, activation='relu',  name='hidden_layer_2'))  # Capa oculta 2
model.add(tf.keras.layers.Dense(64, activation='relu',  name='hidden_layer_3'))  # Capa oculta 3
model.add(tf.keras.layers.Dense(64, activation='relu',  name='hidden_layer_4'))  # Capa oculta 4

# Output layer
model.add(tf.keras.layers.Dense(1, activation='linear', name='output_layer'))


model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_layer_1 (Dense)      (None, 64)                768       
                                                                 
 hidden_layer_2 (Dense)      (None, 64)                4160      
                                                                 
 hidden_layer_3 (Dense)      (None, 64)                4160      
                                                                 
 hidden_layer_4 (Dense)      (None, 64)                4160      
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 13,313
Trainable params: 13,313
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compilación del modelo
# Código aquí

model.compile(optimizer='adam', loss='mean_squared_error')  # Compilación del modelo


In [20]:
# No modifique el código
model.fit(x_train,
          y_train,
          epochs=200,
          batch_size=32,
          validation_split=0.2,
          verbose=1)

Epoch 1/200
122/122 [==============================] - 2s 4ms/step - loss: 6.2605 - val_loss: 1.8358
Epoch 2/200
122/122 [==============================] - 0s 3ms/step - loss: 1.1887 - val_loss: 1.1442
Epoch 3/200
122/122 [==============================] - 0s 2ms/step - loss: 0.7674 - val_loss: 0.7550
Epoch 4/200
122/122 [==============================] - 0s 3ms/step - loss: 0.5976 - val_loss: 0.6019
Epoch 5/200
122/122 [==============================] - 0s 3ms/step - loss: 0.5153 - val_loss: 0.5950
Epoch 6/200
122/122 [==============================] - 0s 2ms/step - loss: 0.5020 - val_loss: 0.5309
Epoch 7/200
122/122 [==============================] - 0s 3ms/step - loss: 0.4775 - val_loss: 0.5968
Epoch 8/200
122/122 [==============================] - 0s 3ms/step - loss: 0.4571 - val_loss: 0.5226
Epoch 9/200
122/122 [==============================] - 0s 3ms/step - loss: 0.4467 - val_loss: 0.5489
Epoch 10/200
122/122 [==============================] - 0s 3ms/step - loss: 0.4368 - val_lo

In [21]:
# No modifique el código
results = model.evaluate(x_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

51/51 [==============================] - 0s 1ms/step - loss: 4192.7583
Test Loss: 4192.75830078125


<a name='1.2'></a>
## Cuestión 2: Utilice el mismo modelo de la cuestión anterior pero añadiendo al menos dos técnicas distinas de regularización.

Ejemplos de regularización: [Prevent_Overfitting.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/Fundamentals/Prevent_Overfitting.ipynb)

Puntuación:

- Obtener el modelo con la regularización: 0.8 pts
- Obtener un `test loss` inferior al anterior: 0.2 pts


In [24]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(11,),  name='hidden_layer_1'))
# Noise injection
model.add(layers.GaussianNoise(2))


# L2 regularization
kerner_regularizer_l2 = regularizers.l2(1e-5)


model.add(tf.keras.layers.Dense(63, kernel_regularizer=kerner_regularizer_l2, activation="tanh", name="hidden_layer_2"))


# Batch Normalization
model.add(layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))




model.add(tf.keras.layers.Dense(64, activation='tanh', name="hidden_layer_3"))

model.add(tf.keras.layers.Dense(64, activation='relu', name="hidden_layer_4"))

# Output layer
model.add(tf.keras.layers.Dense(1, activation='linear', name='output_layer'))


model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_layer_1 (Dense)      (None, 64)                768       
                                                                 
 gaussian_noise (GaussianNoi  (None, 64)               0         
 se)                                                             
                                                                 
 hidden_layer_2 (Dense)      (None, 63)                4095      
                                                                 
 batch_normalization (BatchN  (None, 63)               252       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 63)                0         
                                                                 
 hidden_layer_3 (Dense)      (None, 64)               

In [25]:
# Compilación del modelo
# Código aquí
model.compile(optimizer='adam', loss="mse", metrics=['mae'])

In [26]:
batch_size=15

In [27]:
# No modifique el código
model.fit(x_train,
          y_train,
          epochs=200,
          batch_size=batch_size,
          validation_split=0.2,
          verbose=1)

Epoch 1/200
260/260 [==============================] - 3s 4ms/step - loss: 2.4645 - mae: 1.0495 - val_loss: 6.6864 - val_mae: 2.4232
Epoch 2/200
260/260 [==============================] - 1s 4ms/step - loss: 0.7898 - mae: 0.7030 - val_loss: 3.9914 - val_mae: 1.7965
Epoch 3/200
260/260 [==============================] - 1s 4ms/step - loss: 0.7647 - mae: 0.6881 - val_loss: 2.9365 - val_mae: 1.4885
Epoch 4/200
260/260 [==============================] - 1s 3ms/step - loss: 0.7312 - mae: 0.6732 - val_loss: 2.3184 - val_mae: 1.2780
Epoch 5/200
260/260 [==============================] - 1s 5ms/step - loss: 0.7049 - mae: 0.6597 - val_loss: 1.8008 - val_mae: 1.0990
Epoch 6/200
260/260 [==============================] - 1s 3ms/step - loss: 0.6906 - mae: 0.6522 - val_loss: 1.8200 - val_mae: 1.1060
Epoch 7/200
260/260 [==============================] - 1s 3ms/step - loss: 0.6547 - mae: 0.6364 - val_loss: 1.9484 - val_mae: 1.1377
Epoch 8/200
260/260 [==============================] - 1s 3ms/step - 

In [28]:
# No modifique el código
results = model.evaluate(x_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

51/51 [==============================] - 0s 2ms/step - loss: 1.3091 - mae: 0.9125
Test Loss: [1.3090698719024658, 0.9125260710716248]


<a name='1.3'></a>
## Cuestión 3: Utilice el mismo modelo de la cuestión anterior pero añadiendo un callback de early stopping.

In [29]:
model = tf.keras.models.Sequential()
# Hidden layers

model.add(tf.keras.layers.Dense(64, input_shape=(11, ), activation='relu', name='hidden_layer_1'))
# Noise injection
model.add(layers.GaussianNoise(2))

# L2 regularization
kerner_regularizer_l2 = regularizers.l2(1e-5)

model.add(layers.Dense(63, kernel_regularizer=kerner_regularizer_l2, activation="tanh", name="hidden_layer_2"))

# Batch Normalization
model.add(layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

model.add(layers.Dense(64, activation='tanh', name="hidden_layer_3"))

model.add(layers.Dense(64, activation='relu', name="hidden_layer_4"))

# Output layer
model.add(layers.Dense(1, activation='linear', name='output_layer'))

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_layer_1 (Dense)      (None, 64)                768       
                                                                 
 gaussian_noise_1 (GaussianN  (None, 64)               0         
 oise)                                                           
                                                                 
 hidden_layer_2 (Dense)      (None, 63)                4095      
                                                                 
 batch_normalization_1 (Batc  (None, 63)               252       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 63)                0         
                                                                 
 hidden_layer_3 (Dense)      (None, 64)               

In [30]:
# Compilación del modelo
# Código aquí
model.compile(optimizer='adam', loss="mse", metrics=['mae'])

In [31]:
## definir el early stopping callback
# Código aquí
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss',  patience=15, verbose=1, restore_best_weights=True)
model.fit(x_train,
          y_train,
          epochs=200,
          batch_size=32,
          validation_split=0.2,
          verbose=1,
          callbacks=[early_stopping_callback]) # Código aquí

Epoch 1/200
122/122 [==============================] - 2s 7ms/step - loss: 5.3637 - mae: 1.5728 - val_loss: 8.2640 - val_mae: 2.7251
Epoch 2/200
122/122 [==============================] - 1s 5ms/step - loss: 0.8091 - mae: 0.7091 - val_loss: 6.0925 - val_mae: 2.3023
Epoch 3/200
122/122 [==============================] - 1s 4ms/step - loss: 0.7732 - mae: 0.6876 - val_loss: 4.9599 - val_mae: 2.0322
Epoch 4/200
122/122 [==============================] - 1s 5ms/step - loss: 0.7421 - mae: 0.6786 - val_loss: 4.0133 - val_mae: 1.7679
Epoch 5/200
122/122 [==============================] - 0s 3ms/step - loss: 0.7116 - mae: 0.6602 - val_loss: 3.6356 - val_mae: 1.6435
Epoch 6/200
122/122 [==============================] - 0s 3ms/step - loss: 0.7098 - mae: 0.6615 - val_loss: 2.8409 - val_mae: 1.4169
Epoch 7/200
122/122 [==============================] - 0s 3ms/step - loss: 0.6840 - mae: 0.6466 - val_loss: 2.9355 - val_mae: 1.4392
Epoch 8/200
122/122 [==============================] - 0s 3ms/step - 

In [32]:
# No modifique el código
results = model.evaluate(x_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

51/51 [==============================] - 0s 2ms/step - loss: 1.0933 - mae: 0.8405
Test Loss: [1.0933414697647095, 0.840474009513855]


<a name='1.4'></a>
## Cuestión 4: ¿Podría haberse usado otra función de activación de la neurona de salida? En caso afirmativo especifíquela.

Sí, se podría utilizar la función de activación ReLU que mapea los valores negativos a cero y mantiene los valores positivos sin cambios.

Sin embargo, es importante tener en cuenta que la función de activación ReLU no restringe la salida a un rango específico. Por lo tanto, si se espera que la calidad del vino esté en un rango decimal específico (por ejemplo, 0-10), se debe tener cuidado al interpretar los resultados, ya que la función de activación ReLU podría producir valores fuera de ese rango.

<a name='1.5'></a>
## Cuestión 5:  ¿Qué es lo que una neurona calcula?

**a)** Una función de activación seguida de una suma ponderada  de las entradas.

**b)** Una suma ponderada  de las entradas seguida de una función de activación.

**c)** Una función de pérdida, definida sobre el target.

**d)** Ninguna  de las anteriores es correcta


La respuesta correcta es la:

b)

<a name='1.6'></a>
## Cuestión 6:  ¿Cuál de estas funciones de activación no debería usarse en una capa oculta (hidden layer)?

**a)** `sigmoid`

**b)** `tanh`

**c)** `relu`

**d)** `linear`


La respuesta correcta es la:

d)

<a name='1.7'></a>
## Cuestión 7:  ¿Cuál de estas técnicas es efectiva para combatir el overfitting en una red con varias capas ocultas? Ponga todas las que lo sean.

**a)** Dropout

**b)** Regularización L2.

**c)** Aumentar el tamaño del test set.

**d)** Aumentar el tamaño del validation set.

**e)** Reducir el número de capas de la red.

**f)** Data augmentation.

Las respuestas correctas son:

a)

b)

e)

f)

<a name='1.8'></a>
## Cuestión 8:  Supongamos que queremos entrenar una red para un problema de clasificación de imágenes con las siguientes clases: {'perro','gato','persona'}. ¿Cuántas neuronas y que función de activación debería tener la capa de salida? ¿Qué función de pérdida (loss function) debería usarse?


 Para un problema de clasificación de imágenes con las clases propuestas, la capa de salida debería tener 3 neuronas con una función de activación softmax, y la función de pérdida adecuada sería la entropía cruzada categórica (categorical cross-entropy loss).

<a name='actividad_2'></a>
# Actividad 2: Redes Convolucionales

Vamos a usar el dataset [cifar-10](https://www.cs.toronto.edu/~kriz/cifar.html), que son 60000 imágenes de 32x32 a color  con 10 clases diferentes. Para realizar mejor la práctica puede consultar [Introduction_to_CNN.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/CNN/Introduction_to_CNN.ipynb).



**Puntuación**:

- [Cuestión 1](#2.1): 1 pt
- [Cuestión 2](#2.2): 1.5 pt
- [Cuestión 3](#2.3): 0.5 pts
- [Cuestión 4](#2.4): 0.25 pts
- [Cuestión 5](#2.5): 0.25 pts
- [Cuestión 6](#2.6): 0.25 pts
- [Cuestión 7](#2.7): 0.25 pts




Puede normalizar las imágenes al principio o usar la capa [Rescaling](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Rescaling):

```python
tf.keras.layers.experimental.preprocessing.Rescaling(
    scale, offset=0.0, name=None, **kwargs
)
```

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = y_train.flatten()
y_test = y_test.flatten()

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i])
    plt.xlabel(class_names[y_train[i]])
plt.show()

In [ ]:
print('x_train, y_train shapes:', x_train.shape, y_train.shape)
print('x_test, y_test shapes:', x_test.shape, y_test.shape)

<a name='2.1'></a>
## Cuestión 1: Cree una red convolucional con la API funcional con al menos dos capas convolucionales y al menos dos capas de pooling. Utilize sólo [Average Pooling](https://www.tensorflow.org/api_docs/python/tf/keras/layers/AveragePooling2D) y no añada ninguna regularización.

In [ ]:
inputs = tf.keras.Input(shape=..., name='input')
# reescaling = ...

# Convolution + pooling layers
...

# Flattening
...

# Fully-connected
outputs = layers.Dense(...)

model = keras.Model(inputs=..., outputs=...)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=25, batch_size=64,
                    validation_split=0.15)

In [ ]:
results = model.evaluate(x_test, y_test, verbose=0, batch_size=1000)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

<a name='2.2'></a>
## Cuestión 2: Cree un modelo con la API funcional con un máximo de 2 capas convolucionales y un máximo de 2 capas de pooling. Utilize  [Max Pooling](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) o [Average Pooling](https://www.tensorflow.org/api_docs/python/tf/keras/layers/AveragePooling2D) y  añada la regularización que quiera. Debe obtener un `Test accuracy > 0.68`

In [ ]:
inputs = tf.keras.Input(shape=..., name='input')
# reescaling = ...

# Convolution + pooling layers
...

# Flattening
...

# Fully-connected
outputs = layers.Dense(...)

model = keras.Model(inputs=..., outputs=...)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=..., batch_size=...,
                    validation_split=0.15, callbacks=[...])

In [ ]:
results = model.evaluate(x_test, y_test, verbose=0, batch_size=1000)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

<a name='2.3'></a>
## Cuestión 3: Añada data augmentation al principio del modelo. No es necesario mejorar el test_loss.



In [ ]:
data_augmentation=...

In [ ]:
inputs = tf.keras.Input(shape=..., name='input')
data_aug= ...

# reescaling = ...

# Convolution + pooling layers
...

# Flattening
...

# Fully-connected
outputs = layers.Dense(...)

model = keras.Model(inputs=..., outputs=...)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=..., batch_size=...,
                    validation_split=0.15, callbacks=lbacks=[...])

In [ ]:
results = model.evaluate(x_test, y_test, verbose=0, batch_size=1000)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

<a name='2.4'></a>
## Cuestión 4: Cree el mismo  modelo de manera secuencial. No es necesario compilar ni entrenar el modelo

In [ ]:
model_seq = tf.keras.models.Sequential()
# Código aquí
...

<a name='2.5'></a>
## Cuestión 5: Si tenenemos una  una imagen de entrada de 300 x 300 a color (RGB) y queremos usar una red densa. Si la primera capa oculta tiene 100 neuronas, ¿Cuántos parámetros tendrá esa capa (sin incluir el bias) ?


<a name='2.6'></a>
## Cuestión 6   Ponga  las verdaderas ventajas de las redes convolucionales respecto a las densas

**a)** Reducen el número total de parámetros, reduciendo así el overfitting.

**b)** Permiten utilizar una misma 'función'  en varias localizaciones de la imagen de entrada, en lugar de aprender una función diferente para cada pixel.

**c)** Permiten el uso del transfer learning.

**d)** Generalmente son menos profundas, lo que facilita su entrenamiento.



<a name='2.7'></a>
## Cuestión 7: Para el procesamiento de series temporales las redes convolucionales no son efectivas, habrá que usar redes recurrentes.

- **Verdadero**
- **Falso**

<a name='actividad_3'></a>
# Actividad 3: Redes Recurrentes


- [Cuestión 1](#3.1): 0.5 pt
- [Cuestión 2](#3.2): 0.5 pt
- [Cuestión 3](#3.3): 0.5 pts
- [Cuestión 4](#3.4): 0.25 pts
- [Cuestión 5](#3.5): 0.25 pts

Vamos a usar un dataset de las temperaturas mínimas diarias en Melbourne. La tarea será la de predecir la temperatura mínima en dos días. Puedes usar técnicas de series temporales vistas en otras asignaturas, pero no es necesario.


In [ ]:
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)

In [ ]:
df = pd.read_csv(data_dir, parse_dates=['Date'])
df.head()

In [ ]:
temperatures = df['Temp'].values
print('number of samples:', len(temperatures))
train_data = temperatures[:3000]
test_data = temperatures[3000:]
print('number of train samples:', len(train_data))
print('number of test samples:', len(test_data))
print('firsts trainn samples:', train_data[:10])

<a name='3.1'></a>
## Cuestión 1: Convierta `train_data` y `test_data`  en ventanas de tamaño 5, para predecir el valor en 2 días

En la nomenclatura de [Introduction_to_RNN_Time_Series.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)
```python
win_size, horizon = (5, 2)
```

Para las primeras 10 muestras de train_data `[20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ]` el resultado debería ser:

```python
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.             
```

In [ ]:
# windowing function

In [ ]:
win_size, horizon = (5, 2)

# Puede usar np
# X_train, y_train = ...
# X_test, y_test = ...
# Puede usar tf.data datasets también
# dataset_train = ...
# dataset_test = ...

<a name='3.2'></a>
## Cuestión 2: Cree un modelo recurrente de dos capas GRU para predecir con las ventanas de la cuestión anterior.


In [ ]:
inputs = keras.layers.Input(shape=(..., ...))
...
model = keras.Model(inputs=..., outputs=...)
model.compile(...)
model.summary()

In [ ]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    ...,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

In [ ]:
results = model.evaluate(..., verbose=1)
print('Test Loss: {}'.format(results))

<a name='3.3'></a>
## Cuestión 3: Añada más features a la series temporal, por ejemplo `portion_year`. Cree un modelo que mejore al anterior.


In [ ]:
## Puede añadir más features
df['portion_year'] = df['Date'].dt.dayofyear / 365.0
df_multi = df[['Temp', 'portion_year']].copy()

## train - test split
train_data = df_multi.iloc[:3000].copy()
test_data = df_multi.loc[3000:, :].copy()

In [ ]:
## Create windows
# Puede usar numpy
# X_train, y_train = ...
# X_test, y_test = ...

# Pueden usar tf.data datasets también
# dataset_train = ...
# dataset_test = ...

In [ ]:
inputs = keras.layers.Input(shape=(..., ...))
...
model = keras.Model(inputs=..., outputs=...)
model.compile(...)
model.summary()

In [ ]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)


history = model.fit(
    ...,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

# si ha usado datasets:
# history = model.fit(dataset_train, epochs=200, validation_data=..., callbacks=[...])

In [ ]:
results = model.evaluate(..., verbose=1)
print('Test Loss: {}'.format(results))

<a name='3.4'></a>
## Cuestión 4: ¿En cuáles de estas aplicaciones se usaría un arquitectura 'many-to-one'?

**a)** Clasificación de sentimiento en textos

**b)** Verificación de voz para iniciar el ordenador.

**c)** Generación de música.

**d)** Un clasificador que clasifique piezas de música según su autor.


<a name='3.5'></a>
## Cuestión 5: ¿Qué ventajas aporta el uso de word embeddings?

**a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.

**b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.

**c)** Son una manera de realizar transfer learning en nlp.

**d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.
